In [ ]:
# =====================================================
# FAKE NEWS DETECTION – MINILM + LoRA (PEFT) x FAKENEWSNET
# =====================================================

# 1. CÀI ĐẶT
!pip install -q transformers datasets peft accelerate bitsandbytes scikit-learn pandas numpy psutil

import os, re, shutil, psutil, warnings
import pandas as pd
import numpy as np
import torch
from datasets import load_dataset, Dataset, DatasetDict, concatenate_datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score
from sklearn.utils.class_weight import compute_class_weight
from transformers import (
    AutoTokenizer, AutoModelForSequenceClassification,
    Trainer, TrainingArguments, EarlyStoppingCallback,
    DataCollatorWithPadding
)
from peft import LoraConfig, get_peft_model, TaskType
from google.colab import drive

warnings.filterwarnings("ignore")

# 2. KIỂM TRA GPU
if torch.cuda.is_available():
    device_name = torch.cuda.get_device_name(0)
    print(f"✅ Device: {device_name} | VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    print("⚠️ CẢNH BÁO: Đang chạy trên CPU! Hãy chuyển sang T4 GPU để train LoRA.")

# 3. MOUNT DRIVE (Fix lỗi mount cũ)
drive.mount('/content/drive', force_remount=True)
OUTPUT_DIR = "/content/drive/MyDrive/FakeNewsNet_MiniLM_LoRA"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# 4. TẢI DATASET: FAKENEWSNET
print("\n⏳ Đang tải dataset FakeNewsNet...")
try:
    ds_gossip = load_dataset("rickstello/FakeNewsNet", "gossipcop", split="train")
    ds_politi = load_dataset("rickstello/FakeNewsNet", "politifact", split="train")
    dataset_full = concatenate_datasets([ds_gossip, ds_politi])
    df = pd.DataFrame(dataset_full)
except Exception as e:
    print(f"⚠️ Tải config thất bại ({e}), tải bản default...")
    dataset = load_dataset("rickstello/FakeNewsNet", split="train")
    df = pd.DataFrame(dataset)

print(f"Tổng số mẫu: {len(df)}")

# 5. XỬ LÝ CỘT (Auto-detect)
text_col = next((c for c in ['news_content', 'text', 'content', 'body'] if c in df.columns), None)
title_col = next((c for c in ['title', 'news_title', 'headline'] if c in df.columns), None)
label_col = next((c for c in ['real', 'label', 'class', 'fake'] if c in df.columns), None)

print(f"Mapping: Text='{text_col}' | Title='{title_col}' | Label='{label_col}'")
if not label_col: raise ValueError("❌ Không tìm thấy cột nhãn!")

df['label'] = df[label_col]

# Ghép Title + Text
title_data = df[title_col].fillna('') if title_col else ""
text_data = df[text_col].fillna('') if text_col else ""
df['content'] = title_data + " [SEP] " + text_data

# Làm sạch
def clean_text(s):
    if not isinstance(s, str): return ""
    s = s.lower()
    s = re.sub(r'https?://\S+', ' ', s)
    s = re.sub(r'<.*?>', ' ', s)
    s = re.sub(r'[^a-z0-9\s]', ' ', s)
    s = re.sub(r'\s+', ' ', s).strip()
    return s

print("🧹 Đang làm sạch...", end="")
df['content'] = df['content'].apply(clean_text)
df = df[df['content'].str.len() > 20].drop_duplicates(subset=['content'])
print(f" → Sau xử lý: {len(df):,}")

# Class Weights
classes = np.unique(df['label'])
class_weights = compute_class_weight('balanced', classes=classes, y=df['label'])
class_weight_dict = {k: float(v) for k, v in zip(classes, class_weights)}
print("Class weights:", class_weight_dict)

# 6. SPLIT DATA
train_df, temp_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['label'])
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42, stratify=temp_df['label'])

dataset_dict = DatasetDict({
    "train": Dataset.from_pandas(train_df[['content','label']].reset_index(drop=True)),
    "validation": Dataset.from_pandas(val_df[['content','label']].reset_index(drop=True)),
    "test": Dataset.from_pandas(test_df[['content','label']].reset_index(drop=True))
})

# 7. TOKENIZER
MODEL_NAME = "nreimers/MiniLM-L6-H384-uncased"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

def tokenize_fn(batch):
    return tokenizer(batch["content"], truncation=True, max_length=384, padding=False)

tokenized = dataset_dict.map(tokenize_fn, batched=True, remove_columns=['content'])
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# 8. LOAD MODEL & CONFIG LoRA
print(f"\n⚙️ Tải model: {MODEL_NAME} và áp dụng LoRA...")
base_model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)

# Config nhãn (0: Fake, 1: Real - giả định dataset FakeNewsNet gốc)
base_model.config.id2label = {0: "Fake", 1: "Real"}
base_model.config.label2id = {"Fake": 0, "Real": 1}

# CẤU HÌNH LoRA QUAN TRỌNG
lora_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    r=16,                  
    lora_alpha=32,         
    lora_dropout=0.05,
    bias="none",
    # MiniLM dựa trên kiến trúc BERT, các module attention thường là query, key, value
    target_modules=["query", "key", "value", "dense"]
)

model = get_peft_model(base_model, lora_config)
model.print_trainable_parameters()

# 9. CUSTOM TRAINER
class WeightedTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        w = torch.tensor(list(class_weight_dict.values()), dtype=torch.float32, device=model.device)
        loss_fct = torch.nn.CrossEntropyLoss(weight=w)
        loss = loss_fct(logits, labels)
        return (loss, outputs) if return_outputs else loss

# 10. TRAINING ARGS
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    num_train_epochs=5,                 
    per_device_train_batch_size=32,    
    per_device_eval_batch_size=64,
    gradient_accumulation_steps=1,
    learning_rate=1e-3,                 
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    fp16=torch.cuda.is_available(),
    report_to="none"
)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="weighted")
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}

trainer = WeightedTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)

# 11. TRAIN & EVAL
print("\n🚀 BẮT ĐẦU HUẤN LUYỆN (MiniLM + LoRA)...")
trainer.train()

print("\n🎯 KẾT QUẢ TRÊN TEST SET:")
print(trainer.evaluate(tokenized["test"]))

# Save Adapter
final_path = os.path.join(OUTPUT_DIR, "final_minilm_lora_fnn")
trainer.save_model(final_path)
tokenizer.save_pretrained(final_path)
print(f"\n✅ Đã lưu LoRA Adapter tại: {final_path}")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 19.8 MB/s eta 0:00:00
✅ Device: Tesla T4 | VRAM: 15.8 GB
Mounted at /content/drive

⏳ Đang tải dataset FakeNewsNet...


README.md:   0%|          | 0.00/23.0 [00:00<?, ?B/s]

⚠️ Tải config thất bại (BuilderConfig 'gossipcop' not found. Available: ['default']), tải bản default...


FakeNewsNet.csv: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/23196 [00:00<?, ? examples/s]

...

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

trainable params: 676,610 || all params: 23,390,596 || trainable%: 2.8927

🚀 BẮT ĐẦU HUẤN LUYỆN (MiniLM + LoRA)...


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.567200,0.494281,0.798027,0.805085,0.818317,0.798027
2,0.480400,0.504557,0.776421,0.787678,0.814513,0.776421
3,0.424200,0.500717,0.803664,0.812580,0.833751,0.803664
4,0.379700,0.496738,0.829967,0.834008,0.840744,0.829967
5,0.331500,0.517316,0.831376,0.836006,0.844385,0.831376



🎯 KẾT QUẢ TRÊN TEST SET:


{'eval_loss': 0.5160755515098572, 'eval_accuracy': 0.8107092531705026, 'eval_f1': 0.817201571826532, 'eval_precision': 0.829754115978333, 'eval_recall': 0.8107092531705026, 'eval_runtime': 0.7842, 'eval_samples_per_second': 2715.041, 'eval_steps_per_second': 43.359, 'epoch': 5.0}

✅ Đã lưu LoRA Adapter tại: /content/drive/MyDrive/FakeNewsNet_MiniLM_LoRA/final_minilm_lora_fnn
